In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [20]:
DATA_DIR = '/home/jeff/Downloads/data/statoil/'
train = pd.read_json(os.path.join(DATA_DIR, 'train.json'))
train['inc_angle'] = pd.to_numeric(train['inc_angle'],errors='coerce')
train.fillna(method='pad', inplace=True)

In [21]:
# stat collection from https://www.kaggle.com/muonneutrino/exploration-transforming-images-in-python
def get_stats(train,label=1):
    train['max'+str(label)] = [np.max(np.array(x)) for x in train['band_'+str(label)] ]
    train['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in train['band_'+str(label)] ]
    train['min'+str(label)] = [np.min(np.array(x)) for x in train['band_'+str(label)] ]
    train['minpos'+str(label)] = [np.argmin(np.array(x)) for x in train['band_'+str(label)] ]
    train['med'+str(label)] = [np.median(np.array(x)) for x in train['band_'+str(label)] ]
    train['std'+str(label)] = [np.std(np.array(x)) for x in train['band_'+str(label)] ]
    train['mean'+str(label)] = [np.mean(np.array(x)) for x in train['band_'+str(label)] ]
    train['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in train['band_'+str(label)] ]
    train['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in train['band_'+str(label)] ]
    train['mid50_'+str(label)] = train['p75_'+str(label)]-train['p25_'+str(label)]

    return train
train = get_stats(train,1)
train = get_stats(train,2)

In [18]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg,max1,maxpos1,min1,minpos1,med1,...,max2,maxpos2,min2,minpos2,med2,std2,mean2,p25_2,p75_2,mid50_2
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0,-0.213149,2735,-38.211376,2404,-27.879921,...,-11.252153,2810,-41.135918,3125,-30.007847,2.381284,-29.910117,-31.591387,-28.267622,3.323765
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0,12.570409,2956,-23.125309,3841,-13.654199,...,0.044052,2957,-34.765831,4349,-25.486130,2.934098,-25.359106,-26.963490,-23.997253,2.966237
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1,-9.918477,2736,-33.391197,2268,-22.935019,...,-15.605879,2661,-34.148819,1570,-24.606205,2.125275,-24.839821,-26.033436,-23.379602,2.653834
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0,4.795627,2664,-32.204136,1637,-23.303238,...,-5.554516,2664,-39.564053,1346,-29.549923,2.389980,-29.567913,-31.042850,-28.276203,2.766647
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0,-6.956036,2585,-35.010487,4401,-25.206615,...,-9.434006,2510,-40.276115,3815,-25.466921,2.276244,-25.627726,-26.895178,-24.017248,2.877930


In [24]:
model = LogisticRegression()
model.fit(train[['inc_angle', 'max1', 'min1', 'med1', 'std1', 'mean1', 'p25_1', 'p75_1',
                'mid50_1', 'max2', 'min2', 'med2', 'std2', 'mean2', 'p25_2', 'p75_2']], train['is_iceberg'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
test = pd.read_json(os.path.join(DATA_DIR, 'test.json'))

In [26]:
test = get_stats(test, 1)
test = get_stats(test, 2)

In [36]:
preds = model.predict_proba(test[['inc_angle', 'max1', 'min1', 'med1', 'std1', 'mean1', 'p25_1', 'p75_1',
                'mid50_1', 'max2', 'min2', 'med2', 'std2', 'mean2', 'p25_2', 'p75_2']])

In [40]:
 with open(os.path.join(DATA_DIR, 'predictions.csv'), 'w') as pred_file:
        pred_file.write('id,is_iceberg\n')
        for i,pred in enumerate(preds):
            pred_file.write('{},{}\n'.format(test['id'][i], pred[1]))

In [39]:
model.classes_

array([0, 1])